In [9]:
import pandas as pd
import numpy as np
import os

WD_newcount = pd.read_csv('/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision Work/WD LC/Updated_WD2014 Layer Count.tab', sep='\t', comment='#', names=['depth', 'age'])
WD_oldcount = pd.read_csv('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/WDC/WD2014 Layer Count.tab', usecols=[0,1,2,3], comment='#', sep='\t', names=['depth', 'age', 'age_unc1', 'age_unc2'])
WD_oldcount['age'] = WD_oldcount['age'] - 0.001
WD_oldcount['age'] = WD_oldcount['age'] * 1000
WD_oldcount['age_unc1'] = WD_oldcount['age_unc1'] * 1000
WD_oldcount['age_unc2'] = WD_oldcount['age_unc2'] * 1000

WD_newcount['age_unc'] = WD_oldcount['age_unc1']
WD_newcount['age'] = WD_newcount['age'] * 1000
WD_oldcount = WD_oldcount[12057:]

print(WD_newcount)
print(WD_oldcount)

          depth      age  age_unc
0         0.360    -56.0      0.0
1         0.767    -55.0      1.0
2         1.156    -54.0      1.0
3         1.701    -53.0      1.0
4         2.162    -52.0      1.0
...         ...      ...      ...
12051  2006.625  11995.0     80.6
12052  2006.756  11996.0     80.6
12053  2006.866  11997.0     80.6
12054  2006.942  11998.0     80.7
12055  2007.019  11999.0     80.7

[12056 rows x 3 columns]
          depth      age  age_unc1  age_unc2
12057  2007.085  12000.0      80.7       NaN
12058  2007.183  12001.0      80.8       NaN
12059  2007.281  12002.0      80.8       NaN
12060  2007.369  12003.0      80.8       NaN
12061  2007.434  12004.0      80.8       NaN
...         ...      ...       ...       ...
65530  3392.913  65473.0       NaN     542.4
65531  3392.919  65474.0       NaN     542.5
65532  3392.924  65475.0       NaN     542.5
65533  3392.930  65476.0       NaN     542.6
65534  3392.936  65477.0       NaN     542.7

[53478 rows x 4 columns]


In [10]:
WD_oldcount["sigma"] = WD_oldcount["age_unc1"].combine_first(WD_oldcount["age_unc2"])

In [20]:
WD_newcount.reset_index(drop=True, inplace=True)
gicc21_new = WD_newcount[['depth', 'age', 'age_unc']]
gicc21_new.dropna(inplace=True)
gicc21_new['comment'] = np.nan

#this is here to test for an error
gicc21_new.loc[gicc21_new['age_unc'] <= 0, 'age_unc'] = 0.1

print(gicc21_new)

gicc21_new.to_csv('/Users/quinnmackay/Desktop/temp/ice_age_horizons.txt', index=False, sep="\t")

          depth      age  age_unc  comment
0         0.360    -56.0      0.1      NaN
1         0.767    -55.0      1.0      NaN
2         1.156    -54.0      1.0      NaN
3         1.701    -53.0      1.0      NaN
4         2.162    -52.0      1.0      NaN
...         ...      ...      ...      ...
12051  2006.625  11995.0     80.6      NaN
12052  2006.756  11996.0     80.6      NaN
12053  2006.866  11997.0     80.6      NaN
12054  2006.942  11998.0     80.7      NaN
12055  2007.019  11999.0     80.7      NaN

[12056 rows x 4 columns]


In [13]:
# do the calculation

# Define year intervals
min_age = WD_oldcount["age"].min()
max_age = WD_oldcount["age"].max()
intervals = np.arange(min_age, max_age, 100) #set year intervals, ignoring the end if less than 100 years

depths = np.interp(intervals, WD_oldcount["age"], WD_oldcount["depth"])

sigma_start = np.interp(intervals, WD_oldcount["age"], WD_oldcount["sigma"])
sigma_end = np.interp(intervals+99, WD_oldcount["age"], WD_oldcount["sigma"])

sigma_diff = sigma_end - sigma_start

#this is here to test for an error
for i in range(len(sigma_diff)):
    if sigma_diff[i] <= 0:
        sigma_diff[i] = 0.1

interval_df = pd.DataFrame({
    "depth_top": depths[:-1],
    "depth_bot": depths[1:],
    "duration": 100,
    "dur_unc": sigma_diff[:-1]
})

interval_df.dropna(inplace=True)

if os.name == 'nt':
    output_file = "E:/GitHub/BICC/Paleochrono/EDML-WDC_test/WDC/ice_age_intervals.txt"
elif os.name == 'posix':
    output_file = '/Users/quinnmackay/Desktop/temp/ice_age_intervals.txt'


interval_df.to_csv(output_file, sep="\t", index=False)

print(f"Done! Saved to: {output_file}")

Done! Saved to: /Users/quinnmackay/Desktop/temp/ice_age_intervals.txt
